# Third step in pipeline
# Get topics of predicted segments

In [2]:
import numpy as np

In [46]:
# Load boundaries and probability desiti vectors of corresponding transcript
hdbscan_model_number = 15
transcript_ref = '13NDTKL5ZGs8cb8dojW3bz'

boundaries = np.load(f'../Thesis/evaluation/boundaries_msize{hdbscan_model_number}_{transcript_ref}.npy')
proba_vectors = np.load(f'../Thesis/annotated_probabilities_msize{hdbscan_model_number}/proba_dens_vec_msize{hdbscan_model_number}_{transcript_ref}.npy')
print(len(boundaries))
print(proba_vectors.shape)

10
(224, 2880)


In [47]:
# For each segment, add probability desity vectors and find max index which matches with the topic 
certainty_ls = []

for index, bound in enumerate(boundaries):
    if index == len(boundaries)-1:
        segment_before = proba_vectors[boundaries[index-1]:bound,:]
        segment_after = proba_vectors[bound:,:]
        segment_before_added = np.add.reduce(segment_before)
        segment_after_added = np.add.reduce(segment_after)
        max_topic_before = np.argmax(segment_before_added)  # index of largest element
        max_topic_after = np.argmax(segment_after_added)  # index of largest element
        norm_value_before = np.max(segment_before_added)/len(segment_before)  # value of largest element
        norm_value_after = np.max(segment_after_added)/len(segment_after)  # value of largest element
        certainty_ls.append([norm_value_before, len(segment_before)])
        certainty_ls.append([norm_value_after, len(segment_after)])
        print(f'Segment before position {bound} has topic: {max_topic_before} and normalised value: {norm_value_before:.3f}')
        print(f'Segment after position {bound} has topic: {max_topic_after} and normalised value: {norm_value_after:.3f}')
    elif index == 0:
        segment = proba_vectors[:bound,:]
        segment_added = np.add.reduce(segment)
        max_topic = np.argmax(segment_added)  # index of largest element
        norm_value = np.max(segment_added)/len(segment)  # value of largest element
        certainty_ls.append([norm_value, len(segment)])
        print(f'Segment before position {bound} has topic: {max_topic} and normalised value: {norm_value:.3f}')
    else:
        segment = proba_vectors[boundaries[index-1]:bound,:]
        segment_added = np.add.reduce(segment)
        max_topic = np.argmax(segment_added)  # index of largest element
        norm_value = np.max(segment_added)/len(segment)  # value of largest element
        certainty_ls.append([norm_value, len(segment)])
        print(f'Segment before position {bound} has topic: {max_topic} and normalised value: {norm_value:.3f}')

# Calculate weighted average certainty
certainty_arr = np.array(certainty_ls)
weights = certainty_arr[:,1]/sum(certainty_arr[:,1])
weighted_average = sum(certainty_arr[:,0] * weights)
print(f'Weighted average certainty: {weighted_average:.1%}')

Segment before position 35 has topic: 243 and normalised value: 0.180
Segment before position 65 has topic: 243 and normalised value: 0.264
Segment before position 80 has topic: 1884 and normalised value: 0.066
Segment before position 84 has topic: 1580 and normalised value: 0.240
Segment before position 96 has topic: 243 and normalised value: 0.091
Segment before position 114 has topic: 243 and normalised value: 0.225
Segment before position 134 has topic: 243 and normalised value: 0.082
Segment before position 152 has topic: 243 and normalised value: 0.088
Segment before position 177 has topic: 243 and normalised value: 0.160
Segment before position 188 has topic: 1744 and normalised value: 0.091
Segment after position 188 has topic: 243 and normalised value: 0.111
Weighted average certainty: 15.0%
